# <a style = background:lightgreen;color:black>Settlement Prediction on train_s (modeling) </a>

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('../DATA/final_train_s_dummies.csv')

In [5]:
X = data[['Tax Related', 'Number of Lawyers',
     'Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted', 'Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']]
y=data['Settlement']

## <a style=background:yellow;color:black> Standardization and Split in training, validation and testing set </a>

You need to scale just the variables that are neither dummies nor boolean.

In [6]:
X_to_scale = X[['Number of Lawyers','Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted']]
X_not_to_scale = X[['Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']]

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

y = pd.DataFrame(y)
std_scale = StandardScaler()
X_scaled = std_scale.fit_transform(X_to_scale)

In [8]:
X_scaled_df = pd.DataFrame(X_scaled, columns=[X_to_scale.columns])
X_scaled_df = pd.concat([X_scaled_df, X_not_to_scale], axis=1)

In [9]:
new_var_names = []
for var_name in X_scaled_df.columns:
    if '(' not in var_name:
        new_var_names.append(var_name)
    else:
        new_var_names.append(var_name.replace('(', '').replace(')', '')
        .replace('\'','').replace(',','').replace('"',''))
new_var_names = []
for var_name in X_scaled_df.columns:
    if type(var_name) != tuple:
        new_var_names.append(var_name)
    else:
        new_var_names.append(var_name[0])

X_scaled_df = pd.DataFrame(X_scaled_df.values, columns=new_var_names)

In [10]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_scaled_df, y, random_state=0,
                                                            test_size=0.1)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val,random_state=0,
                                                            test_size=0.2  )

## <a style=background:yellow;color:black id='reg_tree'> REGRESSION-TREE</a>

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [12]:
reg_tree = DecisionTreeRegressor(random_state=42, criterion='absolute_error',
                                min_samples_split=10,
                                max_features='sqrt')

reg_tree.fit(X_train_val,y_train_val)

DecisionTreeRegressor(criterion='absolute_error', max_features='sqrt',
                      min_samples_split=10, random_state=42)

In [15]:
from sklearn.metrics import mean_absolute_error
y_pred = reg_tree.predict(X_test)
residuals = np.ravel(np.array(y_pred)) - np.ravel(np.array(y_test))

pd.DataFrame({'y_true':np.ravel(y_test), 'y_pred':np.ravel(y_pred), 'residuals':np.ravel(residuals)})

,y_true,y_pred,residuals
0,971.67,600.00,-371.67
1,409.63,464.00,54.37
2,7290.49,11659.89,4369.40
3,1.00,3913.24,3912.24
4,1082.64,1777.23,694.59
...,...,...,...
9655,752.05,600.00,-152.05
9656,314.39,317.00,2.61
9657,500.00,1052.00,552.00
9658,135.43,750.00,614.57


In [14]:
print(mean_absolute_error(y_test, y_pred))

867.1266563146997


## <a style=background:yellow;color:black> MARS (Check the R Script)</a>

* One major problem with trees is their high variance. Often a small change in the data can result in a very different series of splits. This is due to the hierarchical nature of the process (if an error occurs in the first split, then all the other splits can just make it worse and worse). A way to reduce this variance is to use Bagging (We've done it with Random Forest).
* The lack of smoothness of the predictions surfaces and the difficulty in capturing additive structure can be a problem for regression tasks. We can solve this problems by using <a style=color:deepskyblue> **MARS** </a> (Multivariate Additive Regression Splines).

In [ ]:
# # This chunk is for the MARS model.
# final_train_s_dummies_std = pd.concat([X_test, X_train_val],axis=0 )
# final_train_s_dummies_std['Settlement'] = pd.concat([y_test, y_train_val], axis = 0)
# final_train_s_dummies_std.to_excel('final_train_s_dummies_std.xlsx')

## <a style=background:yellow;color:black id='Random_Forest'> Random Forest </a>

In [23]:
from sklearn.ensemble import RandomForestRegressor

rnd_reg = RandomForestRegressor(n_estimators=50,
                                #n_jobs=5,
                                max_depth=10,
                                criterion='absolute_error',
                               random_state = 42,
                               min_samples_split=20,
                               min_samples_leaf=20,
                               max_leaf_nodes=None)

In [24]:
rnd_reg.fit(X_train_val, np.array(y_train_val).ravel())

In [ ]:
y_pred = rnd_reg.predict(X_test)

residuals = np.array(y_pred) - np.array(y_test)

print(mean_absolute_error(y_test, y_pred))
print(mean_absolute_error(y_train_val, rnd_reg.predict(X_train_val)))

765.0234303933746
727.9669157159126


In [ ]:
from sklearn.externals import joblib

# joblib.dumb(rnd_red, '../models/rnd_reg_50_estimators.sav')
# rnd_reg = joblib.load('../models/rnd_reg_50_estimators.sav')

Let's destandardize X_train_val and X_test (the reason why we do it is that we want to build a dataset (data_for_plotting.csv) in order to see the results of the models in Tableau considering the original scale of the variables).

In [ ]:
X_train_val_inv_trans = pd.DataFrame(std_scale.inverse_transform(X_train_val[['Number of Lawyers','Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted']]), columns = ['Number of Lawyers','Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted'], index = X_train_val.index)

X_test_inv_trans = pd.DataFrame(std_scale.inverse_transform(X_test[['Number of Lawyers','Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted']]), columns = ['Number of Lawyers','Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted'], index=X_test.index)

X_train_val_not_std = X_train_val[['Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']]

X_test_not_std = X_test[['Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']]

X_train_val_destandardized = pd.concat([X_train_val_inv_trans, X_train_val_not_std],axis=1)
X_test_destandardized = pd.concat([X_test_inv_trans, X_test_not_std],axis=1)

X_train_val_destandardized = pd.DataFrame(np.array(X_train_val_destandardized), columns=[['Number of Lawyers',
     'Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted', 'Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']], index = X_train_val_destandardized.index)

X_test_destandardized = pd.DataFrame(np.array(X_test_destandardized), columns=[['Number of Lawyers',
     'Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted', 'Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']], index = X_test_destandardized.index)

In [ ]:
# y_pred_MARS = np.array(pd.read_excel('y_pred.xlsx')).ravel()

# data_for_plotting = pd.DataFrame({'y_true':np.concatenate([np.array(y_test),np.array(y_train_val)]).ravel(),
#  'y_pred_rnd_for_no_grid':np.concatenate([np.array(rnd_reg.predict(X_test)),np.array(rnd_reg.predict(X_train_val))]),
# 'y_pred_MARS':y_pred_MARS,
#  'Value formatted':np.array(pd.concat([X_test_destandardized['Value formatted'], X_train_val_destandardized['Value formatted']], axis=0)).ravel(),
#  'Unified contribution formatted':np.array(pd.concat([X_test_destandardized['Unified Contribution formatted'], X_train_val_destandardized['Unified Contribution formatted']], axis=0)).ravel(),
# 'train_test':np.concatenate([np.repeat('test',y_test.shape[0]),np.repeat('train',y_train_val.shape[0])])})

# data_for_plotting.to_csv('data_for_plotting.csv')

## <a style=background:yellow;color:black id='xgboost'> XGBoost</a>

In [25]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [70]:
params = {
    'booster': 'gbtree',
    'learning_rate': 0.05,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'eta': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.5,
    'tree_method': 'exact',
    'verbosity': 0,
    'max_depth': 10,
}

# Define the dataset for xgboost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# # Define the learning rate schedule
# def learning_rate_schedule(iteration):
#     if iteration < 100:
#         return 0.1
#     elif 100 <= iteration < 200:
#         return 0.01
#     else:
#         return 0.001

# # Define the callback to update the learning rate
# def update_learning_rate(env):
#     lr = learning_rate_schedule(env.iteration)
#     env.model.params['learning_rate'] = lr

# Define the stopping criteria
early_stopping_rounds = 50

# Train the model with early stopping
# model = xgb.train(params, dtrain, num_boost_round=500, evals=[(dval, 'val')], early_stopping_rounds=early_stopping_rounds, verbose_eval=False, callbacks=[update_learning_rate])
model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'val')], early_stopping_rounds=early_stopping_rounds, verbose_eval=False)


In [72]:
# model.save_model("../models/xgb_train_s.model")

In [73]:
# xgb_model = xgb.Booster()
# xgb_model.load_model('../models/xgb_train_s.model')

dtest = xgb.DMatrix(X_test)
y_pred_test = model.predict(dtest)
print(mean_absolute_error(y_pred_test,y_test))

805.6419785217511


In [75]:
# data_for_plotting = pd.read_csv('../DATA/data_for_plotting.csv')
# dtrain_val = xgb.DMatrix(X_train_val)
# data_for_plotting['y_pred_xgb'] = np.concatenate([np.array(model.predict(dtest)),np.array(model.predict(dtrain_val))])
# data_for_plotting.to_csv('../DATA/data_for_plotting.csv')
# data_for_plotting = pd.read_csv('../DATA/data_for_plotting.csv')
# data_for_plotting.to_excel('../DATA/data_for_plotting.xlsx')

## <a style=background:yellow;color:black id='neural_network'> Neural Network </a>

In [11]:
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.metrics import mean_absolute_error

The following chunk is for reproducibility purposes. It is not necessary to run it. Moreover, keras.backend.clear_session() is useful in cases where you want to start a new session with a fresh set of variables, or if you want to clear the session to free up memory.

In [12]:
tf.random.set_seed(42)
keras.backend.clear_session() 
np.random.seed(42)

In the following chunk we have the learning rate scheduler of the neural network (for the first 3 epochs it is equal to the lr defined in the .compile() method and afterwords at each epoch it decreases of lr*exp(-0.025)). Moreover there is an early stopping callback and a tensorboard callback. The tensorboard callback is used to visualize the training process in Tensorboard. The early stopping callback is used to stop the training process when the validation loss doesn't decrease for 10 epochs. The get_run_logdir function is used to create a unique subdirectory in which to save the Tensorboard log files for each training run. This is necessary to avoid problems when you launch multiple Tensorboard instances at the same time.

The histogram_freq parameter in the TensorBoard callback of Keras determines how often (in number of epochs) to compute activation and weight histograms for the layers of the model. If set to 1, histograms will be computed after every epoch. If set to 0, histograms will not be computed. If set to a value greater than 1, histograms will be computed every histogram_freq epochs.

In [13]:
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.025)

def get_run_logdir(root_logdir, model_name):
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, model_name+'_'+run_id)

model_name = input('Enter model name: ')
root_logdir = os.path.join(os.curdir,'..', 'my_logs')
run_logdir = get_run_logdir(root_logdir, model_name)

lr_schedule_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir,
                                            histogram_freq=1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal",input_shape=X_train.shape[1:]),
    keras.layers.Dense(128, activation='relu', kernel_initializer="he_normal"),
    keras.layers.Dense(64, activation='relu', kernel_initializer="he_normal"),
    keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(1)
])

model.compile(loss='mae', optimizer=keras.optimizers.Adam(learning_rate=0.015))

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),callbacks=[tensorboard_cb, lr_schedule_cb, early_stop_cb])

In [22]:
# model.save('../models/NeuralNet_train_s.h5')

In [ ]:
!tensorboard --logdir=../my_logs --port=6006

In [ ]:
mean_absolute_error(y_test, model.predict(X_test))

In [ ]:
# data_for_plotting = pd.read_excel('data_for_plotting.xlsx')
# data_for_plotting['y_pred_neural_network_oversamp'] = np.concatenate([np.array(model.predict(X_test)),np.array(model.predict(X_train_val))])
# data_for_plotting.to_excel('data_for_plotting.xlsx')

## <a style=background:yellow;color:black> Downsampling and Upsampling </a>

In this chapter we are going to try to solve the unbalance problem with downsampling and upsampling.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('../DATA/final_train_s_dummies.csv')

X_y = data[['Tax Related', 'Number of Lawyers',
     'Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted', 'Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002', 'Settlement']]



In [ ]:
# This chunk takes about 20 seconds to run
ranges_value = []
for i in range(X_y.shape[0]):
    if (X_y.iloc[i]['Value formatted'] >= 0) & (X_y.iloc[i]['Value formatted'] < 1) :
         ranges_value.append('[0-1)')
    elif (X_y.iloc[i]['Value formatted'] >= 1) & (X_y.iloc[i]['Value formatted'] <= 1000) :
         ranges_value.append('[1,1000]')
    elif (X_y.iloc[i]['Value formatted'] > 1000) & (X_y.iloc[i]['Value formatted'] <= 1100) :
         ranges_value.append('(1000,1100]')
    elif (X_y.iloc[i]['Value formatted'] > 1100) & (X_y.iloc[i]['Value formatted'] <= 5200) :
         ranges_value.append('(1100,5200]')
    elif (X_y.iloc[i]['Value formatted'] > 5200) & (X_y.iloc[i]['Value formatted'] <= 26000) :
         ranges_value.append('(5200,26000]')
    elif (X_y.iloc[i]['Value formatted'] > 26000) & (X_y.iloc[i]['Value formatted'] <= 52000) :
         ranges_value.append('(26000,52000]')
    elif (X_y.iloc[i]['Value formatted'] > 52000) & (X_y.iloc[i]['Value formatted'] <= 260000) :
         ranges_value.append('(52000,260000]')
    elif (X_y.iloc[i]['Value formatted'] > 260000) & (X_y.iloc[i]['Value formatted'] <= 520000) :
         ranges_value.append('(260000,520000]')
    elif (X_y.iloc[i]['Value formatted'] > 520000):
         ranges_value.append('(520000,inf)')
    else:
        ranges_value.append('error')

X_y['ranges_value']=ranges_value


In [4]:
X_y.groupby(['ranges_value']).size()

ranges_value
(1000,1100]        5369
(1100,5200]       48556
(26000,52000]        61
(5200,26000]       9539
(52000,260000]        8
(520000,inf)          2
[0-1)               748
[1,1000]          32308
dtype: int64

In [5]:
y = X_y['Settlement']

X_y.drop(['Settlement'], axis=1, inplace=True)

X_to_scale = X_y[['Number of Lawyers','Number of Legal Parties', 'Value formatted',
       'Unified Contribution formatted']]

X_not_to_scale = X_y[['Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
       'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
       'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
       'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
       'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
       'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002', 'ranges_value']]


y = pd.DataFrame(y)
std_scale = StandardScaler()
X_scaled = std_scale.fit_transform(X_to_scale)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_to_scale.columns)
X_scaled_df = pd.concat([X_scaled_df, X_not_to_scale], axis=1)
X_train_val, X_test, y_train_val, y_test = train_test_split(X_scaled_df, y, random_state=0,
                                                            test_size=0.1)
X_test.drop(['ranges_value'], axis=1, inplace=True)
X_y_train_val = pd.concat([X_train_val,y_train_val],axis=1)


/var/folders/j3/6zzd1jr16pb3yxx70_376pmw0000gn/T/ipykernel_6407/3290639181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_y.drop(['Settlement'], axis=1, inplace=True)


In the following chunk we're going to:

*  build a downsampled dataset (the four classes with the highest number of observations will be downsampled to 4831 that is the number of observations of the class with less observations among them);
* build an upsampled dataset (the (5200, 26000] class will be upsampled to 29000 observations). The reason is that for this class we have the highest MAE considering the number of observations per class.

In [ ]:
downsampled_df = pd.DataFrame(columns=X_y_train_val.columns)
upsampled_df = pd.DataFrame(columns=X_y_train_val.columns)

for i in X_y_train_val['ranges_value'].unique():
    if i in ['[1,1000]','(1000,1100]','(1100,5200]']:
        downsampled_df = downsampled_df.append(X_y_train_val[X_y_train_val['ranges_value']==i].sample(4831, replace=True))
        upsampled_df = upsampled_df.append(X_y_train_val[X_y_train_val['ranges_value']==i])
    elif i == '(5200,26000]':
        downsampled_df = downsampled_df.append(X_y_train_val[X_y_train_val['ranges_value']==i].sample(4831, replace=True))
        upsampled_df = upsampled_df.append(X_y_train_val[X_y_train_val['ranges_value']==i].sample(29000, replace=True))
    else:
        downsampled_df = downsampled_df.append(X_y_train_val[X_y_train_val['ranges_value']==i])
        upsampled_df = upsampled_df.append(X_y_train_val[X_y_train_val['ranges_value']==i])

In the following chunk we have the learning rate scheduler of the neural network (for the first 3 epochs it is equal to the lr defined in the .compile and afterwords at each epoch it decreases of lr*exp(-0.025)). Moreover there is an early stopping callback and a tensorboard callback. The tensorboard callback is used to visualize the training process in Tensorboard. The early stopping callback is used to stop the training process when the validation loss doesn't decrease for 10 epochs. The get_run_logdir function is used to create a unique subdirectory in which to save the Tensorboard log files for each training run. This is necessary to avoid problems when you launch multiple Tensorboard instances at the same time.

In [7]:
columns_to_convert = ['Tax Related', 'Milano', 'Bari',
       'Bologna', 'Genova', 'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma',
       "L'Aquila", 'Potenza', 'Perugia', 'Campobasso', 'Firenze', 'Cagliari',
       'Venezia', 'Cosenza', 'Ancona', 'Trieste', 'Aosta', 'OR-140999',
       'OR-145009', 'OR-139999', 'OR-145999', 'OR-130099', 'OR-101003',
       'OR-130121', 'OR-130111', 'OR-130131', 'OR-101002', 'OR-180002',
       'OSA-180002']

for column in columns_to_convert:
    downsampled_df[column] = downsampled_df[column].astype('int64')
    upsampled_df[column] = upsampled_df[column].astype('int64')

The following chunk builds and fit a neural network on the downsampled dataset.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Set random seed and clear the session
tf.random.set_seed(42)
keras.backend.clear_session()

# Callbacks
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.025)

def get_run_logdir(root_logdir, model_name):
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, model_name+'_'+run_id)

model_name = input('Enter model name: ')
root_logdir = os.path.join(os.curdir,'..', 'my_logs')
run_logdir = get_run_logdir(root_logdir, model_name)

lr_schedule_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir,
                                            histogram_freq=1)

# Model design
model1 = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal",input_shape=(37,)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(128, activation='relu', kernel_initializer="he_normal"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(64, activation='relu', kernel_initializer="he_normal"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(1)
])

model1.compile(loss='mae', optimizer=keras.optimizers.Adam(learning_rate=0.015))

# Split in train and validation set
train_downsampled, val_downsampled = train_test_split(downsampled_df, test_size=0.2, random_state=42)

# Fitting
history = model1.fit(train_downsampled.drop(columns=['Settlement','ranges_value']),
                     train_downsampled['Settlement'],
                      epochs=100,
                       validation_data=(val_downsampled.drop(columns=['Settlement','ranges_value']),
                        val_downsampled['Settlement']),
                       callbacks=[tensorboard_cb, lr_schedule_cb, early_stop_cb])

# MAE on the test set
mean_absolute_error(y_test, model1.predict(X_test))

The following chunk builds and fit a neural network on the upsampled dataset.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Set random seed and clear the session
tf.random.set_seed(42)
keras.backend.clear_session()

# Callbacks
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.025)

def get_run_logdir(root_logdir, model_name):
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, model_name+'_'+run_id)

model_name = input('Enter model name: ')
root_logdir = os.path.join(os.curdir,'..', 'my_logs')
run_logdir = get_run_logdir(root_logdir, model_name)

lr_schedule_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir,
                                            histogram_freq=1)

# Model design
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal",input_shape=(37,)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(128, activation='relu', kernel_initializer="he_normal"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(64, activation='relu', kernel_initializer="he_normal"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(1)
])

model.compile(loss='mae', optimizer=keras.optimizers.Adam(learning_rate=0.015))

# Split in train and validation set
train_downsampled, val_downsampled = train_test_split(upsampled_df, test_size=0.2, random_state=42)

# Fitting
history = model.fit(train_downsampled.drop(columns=['Settlement','ranges_value']),
                     train_downsampled['Settlement'],
                      epochs=100,
                       validation_data=(val_downsampled.drop(columns=['Settlement','ranges_value']),
                        val_downsampled['Settlement']),
                       callbacks=[tensorboard_cb, lr_schedule_cb, early_stop_cb])

# MAE on the test set
mean_absolute_error(y_test, model.predict(X_test))

In [ ]:
# !tensorboard --logdir=../my_logs --port=6006

The following chunk is to fill data_for_plotting.csv with the prediction of the upsampling neural network and of the downsampling neural network.

In [ ]:
# data_for_plotting = pd.read_csv('../DATA/data_for_plotting.csv')
# data = pd.read_csv('../DATA/final_train_s_dummies.csv')

# X = data[['Tax Related', 'Number of Lawyers',
#      'Number of Legal Parties', 'Value formatted',
#        'Unified Contribution formatted', 'Milano', 'Bari', 'Bologna', 'Genova',
#        'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
#        'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
#        'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
#        'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
#        'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']]

# y=data['Settlement']

# X_to_scale = X[['Number of Lawyers','Number of Legal Parties', 'Value formatted',
#        'Unified Contribution formatted']]

# X_not_to_scale = X[['Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
#        'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
#        'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
#        'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
#        'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
#        'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']]
       
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

# y = pd.DataFrame(y)
# std_scale = StandardScaler()
# X_scaled = std_scale.fit_transform(X_to_scale)
# X_scaled_df = pd.DataFrame(X_scaled, columns=[X_to_scale.columns])
# X_scaled_df = pd.concat([X_scaled_df, X_not_to_scale], axis=1)
# X_train_val, X_test, y_train_val, y_test = train_test_split(X_scaled_df, y, random_state=0,
#                                                             test_size=0.1)

# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val,random_state=0,
#                                                             test_size=0.2  )

# X_train_val = pd.DataFrame(np.array(X_train_val), columns=[['Number of Lawyers',
#      'Number of Legal Parties', 'Value formatted',
#        'Unified Contribution formatted', 'Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
#        'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
#        'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
#        'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
#        'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
#        'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']], index = X_train_val.index)

# X_test = pd.DataFrame(np.array(X_test), columns=[['Number of Lawyers',
#      'Number of Legal Parties', 'Value formatted',
#        'Unified Contribution formatted', 'Tax Related','Milano', 'Bari', 'Bologna', 'Genova',
#        'Palermo', 'Napoli', 'Torino', 'Trento', 'Roma', "L'Aquila", 'Potenza',
#        'Perugia', 'Campobasso', 'Firenze', 'Cagliari', 'Venezia', 'Cosenza',
#        'Ancona', 'Trieste', 'Aosta','OR-140999', 'OR-145009', 'OR-139999',
#        'OR-145999', 'OR-130099', 'OR-101003', 'OR-130121', 'OR-130111',
#        'OR-130131', 'OR-101002', 'OR-180002', 'OSA-180002']], index = X_test.index)

# data_for_plotting['y_pred_neural_network_upsamp'] = np.concatenate([np.array(model.predict(X_test)),np.array(model.predict(X_train_val))])
# data_for_plotting['y_pred_neural_network_downsamp'] = np.concatenate([np.array(model1.predict(X_test)),np.array(model1.predict(X_train_val))])

# data_for_plotting.to_csv('../DATA/data_for_plotting.csv', index=False)